# Collinear Points

## Background

### Collinear points

Definition of collinearity[1]: In geometry, collinearity of a set of points is the property of their lying on a single line. A set of points with this property is said to be collinear.

![](non-collinear-points.jpg)

Here, points P,Q,R and A,R,B are collinear. However, points A,B,C are non-collinear. For more, refer [2].

1. https://en.wikipedia.org/wiki/Collinearity
2. http://www.mathcaptain.com/geometry/collinear-points.html

### Parameterizing lines
In order to determine whether a set of points all lie on the same line we need a standard way to define (or parametrize) a line.

* One way of defining a line is as the set of points $(x,y)$ such that $y=ax+b$ for some fixed real values $a,b$.
* We call $a$ the **slope** of the line and $b$ is the $y$-intercept which is defined as the value of $y$ when $x=0$.
* This parameterization works for *almost* all lines. It does not work for vertical lines. For those lines we define $a$ to be **infinity** and $b$ to be the $x$ intercept of the line (the line is parallel to the $y$ axis so it does not intercept the $y$ axis (other than if it is the vertical line going through the origin).

To summarize, given two different points $(x_1,y_1) \neq (x_2,y_2)$, we define the parameterization $(a,b)$ as:
* **if $x_1=x_2$: ** $(\mbox{Inf},x_1)$ 
* **Else:** $(a,b)$ such that $y_1=a x_1 +b$ and $y_2=a x_2 +b$.


# Task

Given an input file with an arbitrary set of co-ordinates, your task is to use pyspark library functions and write a program in python3 to find if three or more points are collinear.

For instance, if given these points: `{(1,1), (0,1), (2,2), (3,3), (0,5), (3,4), (5,6), (0,-3), (-2,-2)}`

Sets of collinear points are: `{((-2,-2), (1,1), (2,2), (3,3)), ((0,1), (3,4), (5,6)), ((0,-3), (0,1), (0,5))}`. Note that the ordering of the points in a set or the order of the sets does not matter. 

Note: 
* Every set of collinear points has to have **at least three points** (any pair of points lie on a line).
* There are two types of test cases:
     * **Visible Test cases**: Test cases given to you as a part of the notebook. These tests will help you validate your program and figure out bugs in it if any.
     * **Hidden Test cases**: Test cases that are not given as a part of the notebook, but will be used for grading. Cells in this notebook that have `//Hidden test cases here` are read-only cells containing hidden tests.
     * Any cell that does not require you to submit code cannot be modified. For example: Assert statement unit test cells. Cells that have `# YOUR CODE HERE` are the ONLY ones you will need to alter. 
     * DO NOT change the names of functions. 
     * Remove the `Raise NotImplementedError()` line when you write the definition of your function.


# Description of the Approach

The goal of this assignment is to make you familiar with programming using pyspark. There are many ways to find sets of collinear points from a list of points. For the purposes of this assignment, we shall stick with the below approach:

1. List all pairs of points. You can do that efficiently in spark by computing cartesian product of the list of points with itself. For example, given three points $[(1,0), (2,0), (3,0)]$, we construct a list of nine pairs  
$[((1,0),(1,0)),((1,0), (2,0)),((1,0),(3,0))$  
$((2,0),(1,0)),((2,0), (2,0)),((2,0),(3,0))$  
$((3,0),(1,0)),((3,0), (2,0)),((3,0),(3,0))]$  

2. Remove the pairs in which the same point appears twice such as $((2,0),(2,0))$. After these elimination you end up (for this example) with a list of just six pairs:  
$[((1,0),(2,0)),((1,0),(3,0)),((2,0),(1,0)),((2,0),(3,0)),((3,0),(1,0)),((3,0),(2,0))]$

2. For each pair of points, find the parameterization $(a,b)$ of the line connecting them as described above.

3. Group the pairs according to their parameters. Clearly, if two pairs have the same $(a,b)$ values, all points in the two pairs lie on the same line.

3. Eliminate the groups that contain only one pair (any pair of points defines a line).
4. In each of the remaining groups, unpack the point-pairs to identify the individual points.
Note that if a set of points $(x_1,y_1),\ldots,(x_k,y_k)$ lie on the same line then each point will appear $k-1$ times in the list of point-pairs. You therefore need to transform the list of points into sets to remove duplicates.

5. Output the sets of 3 or more colinear points.

Your task is to implement the described algorithm in Spark. You should use RDD's all the way through and collect the results into the driver only at the end.

In [2]:
spark

res0: org.apache.spark.sql.SparkSession = org.apache.spark.sql.SparkSession@1bf55aa3


# Helper Functions
Here are some helper functions that you are encouraged to use in your implementations. Do not change these functions.


The function `format_result` takes an element of the form shown below in the example. It outputs a tuple of all points that are collinear (shown below).

Input: ((A,slope), [C1,..., Ck]) where each of A, C1, ..., Ck is a point of form (Ax, Ay) and slope is of type float.

**<font color="magenta" size=2>Example Code</font>**
``` python
my_input = (((2, 1), 0.5), [(4, 2), (6, 3)]) 
format_result(my_input)
```
Output: (C1,..., Ck, A) each of A,C1,...,Ck is a point of form (Ax, Ay)

**<font color="blue" size=2>Example Output</font>**
``` python
((4, 2), (6, 3), (2, 1))
```

<font color="red">**Hint : **</font> The above example is given just to provide the input and output format. This function is called a different way in the spark exercise.

In [50]:
// Define a new type
type MyTuple = (((AnyVal, AnyVal), AnyVal), List[(AnyVal,AnyVal)])

defined type alias MyTuple


In [51]:
def format_result(x:MyTuple) = {
    val s = x._2
    s :+ (x._1)._1
}

format_result: (x: MyTuple)List[(AnyVal, AnyVal)]


In [69]:
val my_input: MyTuple= (((2,1),0.5),List((4,2), (6,3)))
format_result(my_input)

my_input: MyTuple = (((2,1),0.5),List((4,2), (6,3)))
res23: List[(AnyVal, AnyVal)] = List((4,2), (6,3), (2,1))


## Exercise 1: `to_tuple`

The function `to_tuple` converts each point of form 'Ax Ay' into a point of form (Ax, Ay) for further processing.

**<font color="magenta" size=2>Example Code</font>**
``` scala
my_input = '2 3'
to_tuple(my_input)
```

**<font color="blue" size=2>Example Output</font>**
``` scala
(2, 3)
```

<font color="red">**Hint : **</font> The above example is given just to provide the input and output format. This function is called a different way in the spark exercise.

In [70]:
val s:String = "2 3"

s: String = 2 3


In [72]:
val s2 = s.split(" ")
(s2(0).toInt, s2(1).toInt)

s2: Array[String] = Array(2, 3)
res25: (Int, Int) = (2,3)


In [73]:
def to_tuple(s: String) = {
    val s2 = s.split(" ")
    (s2(0).toInt, s2(1).toInt)
}

to_tuple: (s: String)(Int, Int)


In [79]:
assert(to_tuple("1 1") == (1,1))

## Exercise 2: `non_duplicates`

The function `non_duplicates` checks if a set of points contains duplicates or not.

Input: Pair (A,B) where A and B are of form (Ax, Ay) and (Bx, By) respectively.

**<font color="magenta" size=2>Example Code</font>**
``` python
my_input = ((0,0),(1,2))
non_duplicates(my_input)
```

Output: Returns True if A != B, False otherwise.

**<font color="blue" size=2>Example Output</font>**
``` python
True
```

<font color="red">**Hint : **</font> The above example is given just to provide the input and output format. This function is called a different way in the spark exercise.

In [80]:
val a = (0,0)
val b = (1,2)

a!=b

a: (Int, Int) = (0,0)
b: (Int, Int) = (1,2)
res28: Boolean = true


In [84]:
def non_duplicates(x: ((AnyVal,AnyVal), (AnyVal, AnyVal))): Boolean ={
    val a = x._1
    val b = x._2
    a != b
}

non_duplicates: (x: ((AnyVal, AnyVal), (AnyVal, AnyVal)))Boolean


In [85]:
assert(non_duplicates((0,0),(1,2))== true)

In [87]:
assert(non_duplicates((0,0),(1,2)).isInstanceOf[Boolean])

In [89]:
assert(non_duplicates((0,0),(0,0))==false)